Train

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten,Input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
# Configure TensorFlow for GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth for the GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs Available: {len(gpus)}")
    except RuntimeError as e:
        print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected. Training will use CPU.")


GPUs Available: 1


In [3]:
tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)

In [5]:
train_generator=train_data_gen.flow_from_directory("./data/train",target_size=(48,48),batch_size=64,color_mode="grayscale",class_mode="categorical")

Found 28709 images belonging to 7 classes.


In [6]:
validation_generator=validation_data_gen.flow_from_directory("./data/train",target_size=(48,48),batch_size=64,color_mode="grayscale",class_mode="categorical")

Found 28709 images belonging to 7 classes.


In [7]:
emotion_model=Sequential()

In [8]:
emotion_model.add(Input(shape=(48, 48, 1)))
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

In [9]:
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

In [10]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation='softmax'))

In [11]:
emotion_model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.0001, decay=1e-6),metrics=["accuracy"])

In [12]:
emotion_model_info=emotion_model.fit(train_generator,steps_per_epoch=28709//64,epochs=50,validation_data=validation_generator,validation_steps=7178//64)

Epoch 1/50
448/448 [==============================] - 74s 146ms/step - loss: 1.8057 - accuracy: 0.2544 - val_loss: 1.7183 - val_accuracy: 0.3267
Epoch 2/50
448/448 [==============================] - 61s 136ms/step - loss: 1.6377 - accuracy: 0.3587 - val_loss: 1.5426 - val_accuracy: 0.4124
Epoch 3/50
448/448 [==============================] - 64s 143ms/step - loss: 1.5292 - accuracy: 0.4087 - val_loss: 1.4566 - val_accuracy: 0.4452
Epoch 4/50
448/448 [==============================] - 59s 132ms/step - loss: 1.4572 - accuracy: 0.4419 - val_loss: 1.3677 - val_accuracy: 0.4805
Epoch 5/50
448/448 [==============================] - 62s 139ms/step - loss: 1.3977 - accuracy: 0.4662 - val_loss: 1.3508 - val_accuracy: 0.4865
Epoch 6/50
448/448 [==============================] - 62s 138ms/step - loss: 1.3456 - accuracy: 0.4906 - val_loss: 1.2758 - val_accuracy: 0.5229
Epoch 7/50
448/448 [==============================] - 62s 138ms/step - loss: 1.3018 - accuracy: 0.5092 - val_loss: 1.2284 - val_ac

In [13]:
# Ensure emotion_model is properly initialized
if emotion_model is not None:
    # Convert model to JSON
    model_json = emotion_model.to_json()
    
    # Save the model JSON
    with open("emotion_model.json", "w") as json_file:
        json_file.write(model_json)
    
    # Save the model weights
    emotion_model.save_weights("emotion_model.weights.h5")
else:
    print("Error: emotion_model is None")
